In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(url, headers=headers)

# --- DIAGNOSTIC CHECK ---
# Check if the request was successful (should be 200)
print(f"Request Status Code: {response.status_code}")

soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {'id': 'constituents'})
if table:
    print("\nSuccess! Found the table with id='constituents'.")
    ticker_list = []
    
    # Get all rows from the table body, skip the header row [1:]
    for row in table.find('tbody').find_all('tr')[1:]: 
        
        # Get all data cells ('td') in the row
        cols = row.find_all('td')
        
        if cols:
            # The ticker is the text in the very first cell (index 0)
            ticker = cols[0].text.strip()
            ticker_list.append(ticker)

    print(f"Total tickers: {len(ticker_list)}")
    # Change "." to "-". This is because the Wikipedia list uses "BRK.B" but yfinance uses "BRK-B"
    ticker_list = [ticker.replace('.', '-') for ticker in ticker_list]
    print(ticker_list)
else:
    print("\nError: Could not find table with id='constituents' even with headers.")
    print("This is strange, the page structure may have changed.")


Request Status Code: 200

Success! Found the table with id='constituents'.
Total tickers: 503
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APP', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK-B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'XYZ', 'BK', 'BA', 'BKNG', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF-B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CPT', 'CPB', 'COF', 'CAH', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'COIN', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTV

In [13]:
factors = ["momentum", "defensive", "quality", "value"]

columns = pd.MultiIndex.from_product(
    [ticker_list, factors],
    names=["ticker", "factor"]
)

dates = pd.date_range(start="2022-02-01", end="2025-12-31", freq="M")

df_global = pd.DataFrame(index=dates, columns=columns, dtype=float)

/var/folders/q5/pzd5gn750_76vhjzjlgj6rfr0000gn/T/ipykernel_62424/4031023577.py:8: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(start="2022-02-01", end="2025-12-31", freq="M")


In [14]:
df_global.head()

ticker          MMM                              AOS                          \
factor     momentum defensive quality value momentum defensive quality value   
2022-02-28      NaN       NaN     NaN   NaN      NaN       NaN     NaN   NaN   
2022-03-31      NaN       NaN     NaN   NaN      NaN       NaN     NaN   NaN   
2022-04-30      NaN       NaN     NaN   NaN      NaN       NaN     NaN   NaN   
2022-05-31      NaN       NaN     NaN   NaN      NaN       NaN     NaN   NaN   
2022-06-30      NaN       NaN     NaN   NaN      NaN       NaN     NaN   NaN   

ticker          ABT            ...    ZBRA            ZBH                    \
factor     momentum defensive  ... quality value momentum defensive quality   
2022-02-28      NaN       NaN  ...     NaN   NaN      NaN       NaN     NaN   
2022-03-31      NaN       NaN  ...     NaN   NaN      NaN       NaN     NaN   
2022-04-30      NaN       NaN  ...     NaN   NaN      NaN       NaN     NaN   
2022-05-31      NaN       NaN  ...     NaN   NaN      NaN       NaN     NaN   
2022-06-30      NaN       NaN  ...     NaN   NaN      NaN       NaN     NaN   

ticker                ZTS                          
factor     value momentum defensive quality value  
2022-02-28   NaN      NaN       NaN     NaN   NaN  
2022-03-31   NaN      NaN       NaN     NaN   NaN  
2022-04-30   NaN      NaN       NaN     NaN   NaN  
2022-05-31   NaN      NaN       NaN     NaN   NaN  
2022-06-30   NaN      NaN       NaN     NaN   NaN  

[5 rows x 2012 columns]

In [44]:
def get_sector_map(tickers):
    """
    Fetches sector information for a list of tickers.
    Note: Fetching info one-by-one can be slow.
    """
    sector_map = {}
    print("Fetching sector data (this may take a moment)...")
    for i, ticker in enumerate(tickers, 1):
        try:
            # In a production environment, cache this data
            info = yf.Ticker(ticker).info
            sector_map[ticker] = info.get('sector', 'Unknown')
            if i % 50 == 0:
                print(f"Processed {i}/{len(tickers)} tickers...")
        except Exception as e:
            print(f"Could not fetch sector for {ticker}: {e}")
            sector_map[ticker] = 'Unknown'
    return sector_map

In [45]:
sector_map = get_sector_map(ticker_list)

Fetching sector data (this may take a moment)...
Processed 50/503 tickers...
Processed 100/503 tickers...
Processed 150/503 tickers...
Processed 200/503 tickers...
Processed 250/503 tickers...
Processed 300/503 tickers...
Processed 350/503 tickers...
Processed 400/503 tickers...
Processed 450/503 tickers...
Processed 500/503 tickers...


In [52]:
def calculate_momentum_factor(tickers, lookback_years=5):
    # 1. FETCH DATA
    print("Downloading price history...")
    print(f"Attempting to download {len(tickers)} tickers...")
    
    try:
        data = yf.download(
            tickers, 
            period=f"{lookback_years}y", 
            interval="1mo", 
            progress=True,
            auto_adjust=True,
            threads=True
        )
        
        # When auto_adjust=True, yfinance returns adjusted prices directly
        # For multiple tickers, columns are multi-level: (Price Type, Ticker)
        # We need to extract just the Close prices
        if isinstance(data.columns, pd.MultiIndex):
            # Multi-ticker case: select 'Close' from the multi-level columns
            if 'Close' in data.columns.get_level_values(0):
                data = data['Close']
            else:
                print("Available columns:", data.columns.get_level_values(0).unique().tolist())
                print("Error: Could not find 'Close' prices in data")
                return None, None, None
        else:
            # Single ticker or already the right format
            if 'Close' in data.columns:
                data = data['Close']
            
    except Exception as e:
        print(f"Error downloading data: {e}")
        return None, None, None
    
    # Drop columns with insufficient history (less than 12 months)
    # We require > 12 non-NaN values
    data = data.dropna(axis=1, thresh=12)
    valid_tickers = data.columns.tolist()
    
    if len(valid_tickers) == 0:
        print("ERROR: No valid tickers with sufficient data!")
        print("This may be due to network issues or yfinance rate limiting.")
        print("Try running the cell again or reduce the number of tickers.")
        return None, None, None
    
    # 2. CALCULATE RAW MOMENTUM
    # Formula: P(t-2) / P(t-12) - 1
    # logic: shift(2) moves the price from 2 months ago to current row
    mom_raw = data.shift(2) / data.shift(12) - 1
    
    # Drop the first 12 months as they will be NaN due to the shift
    mom_raw = mom_raw.dropna(how='all')

    # 3. PREPARE FOR CROSS-SECTIONAL STANDARDIZATION
    # Convert from Wide (Tickers as columns) to Long (Date/Ticker rows)
    df_long = mom_raw.stack().reset_index()
    df_long.columns = ['Date', 'Ticker', 'Raw_Momentum']
    
    # Map Sectors
    sector_map = get_sector_map(valid_tickers)
    df_long['Sector'] = df_long['Ticker'].map(sector_map)

    # 4. WINSORIZATION (Cross-sectional per Date)
    # We clip outliers at the 5th and 95th percentiles for each month
    def winsorize_group(group):
        lower = group.quantile(0.05)
        upper = group.quantile(0.95)
        return group.clip(lower, upper)

    df_long['Mom_Winsorized'] = df_long.groupby('Date')['Raw_Momentum'] \
                                       .transform(winsorize_group)

    # 5. SECTOR Z-SCORES
    # Calculate Z-Score per Date and Sector
    # Formula: (x - mean) / std
    def calc_zscore(group):
        if len(group) < 2: 
            return 0.0 # Neutral score if not enough peers in sector
        sigma = group.std()
        if sigma == 0:
            return 0.0
        return (group - group.mean()) / sigma

    df_long['Z_Momentum'] = df_long.groupby(['Date', 'Sector'])['Mom_Winsorized'] \
                                   .transform(calc_zscore)
    
    # 6. ADD QUINTILES
    # Group by date and assign quintiles (1 = lowest momentum, 5 = highest momentum)
    df_long['Momentum_Quintile'] = df_long.groupby('Date')['Z_Momentum'].transform(
        lambda x: pd.qcut(x, q=5, labels=[1, 2, 3, 4, 5], duplicates='drop')
    )
    
    # 7. CALCULATE FORWARD RETURNS
    # Convert price_data to long format
    price_long = data.stack().reset_index()
    price_long.columns = ['Date', 'Ticker', 'Price']
    price_long = price_long.sort_values(['Ticker', 'Date'])
    
    # Calculate forward return: P(t+1)/P(t) - 1
    price_long['Forward_Return'] = price_long.groupby('Ticker')['Price'].transform(
        lambda x: x.shift(-1) / x - 1
    )
    
    # Merge forward returns with detailed_df
    df_long = df_long.merge(
        price_long[['Date', 'Ticker', 'Forward_Return']],
        on=['Date', 'Ticker'],
        how='left'
    )
    
    # Pivot back to wide format for easy viewing/trading matrix
    final_factor = df_long.pivot(index='Date', columns='Ticker', values='Z_Momentum')
    
    return final_factor, df_long, data

# --- EXECUTION ---

# Run calculation
z_score_matrix, detailed_df, price_data = calculate_momentum_factor(ticker_list)

# Check if calculation was successful
if z_score_matrix is not None and not z_score_matrix.empty:
    # Display quintile distribution
    print("MOMENTUM QUINTILE DISTRIBUTION")
    print("\nQuintile Distribution:")
    print(detailed_df['Momentum_Quintile'].value_counts().sort_index())
    print(f"\nSample of data with quintiles:")
    print(detailed_df[['Date', 'Ticker', 'Z_Momentum', 'Momentum_Quintile']].head(20))
    
    # Calculate backtest statistics
    print("MOMENTUM STRATEGY BACKTEST - Monthly Rebalancing")
    
    quintile_stats = detailed_df.groupby('Momentum_Quintile')['Forward_Return'].agg([
        ('Mean_Monthly_Return', 'mean'),
        ('Median_Monthly_Return', 'median'),
        ('Std_Dev', 'std'),
        ('Count', 'count')
    ])
    
    # Convert to percentage
    quintile_stats['Mean_Monthly_Return'] = quintile_stats['Mean_Monthly_Return'] * 100
    quintile_stats['Median_Monthly_Return'] = quintile_stats['Median_Monthly_Return'] * 100
    quintile_stats['Std_Dev'] = quintile_stats['Std_Dev'] * 100
    
    print("\nAverage Returns by Momentum Quintile (%):")
    print(quintile_stats)
    
    # Calculate spread (Q5 - Q1)
    if 5 in quintile_stats.index and 1 in quintile_stats.index:
        spread = quintile_stats.loc[5, 'Mean_Monthly_Return'] - quintile_stats.loc[1, 'Mean_Monthly_Return']
        print(f"\nMomentum Spread (Q5 - Q1): {spread:.2f}% per month")
        print(f"Annualized Spread: {spread * 12:.2f}%")
    
    # Output the most recent Momentum Z-Scores
    print("Most Recent Momentum Z-Scores (Top 10):")
    latest_date = z_score_matrix.index[-1]
    print(z_score_matrix.loc[latest_date].sort_values(ascending=False).head(10))
else:
    print("\nCalculation failed. Please check the errors above and try again.")


Attempting to download 503 tickers...


[*********************100%***********************]  503 of 503 completed


Fetching sector data (this may take a moment)...
Processed 50/501 tickers...
Processed 100/501 tickers...
Processed 150/501 tickers...
Processed 200/501 tickers...
Processed 250/501 tickers...
Processed 300/501 tickers...
Processed 350/501 tickers...
Processed 400/501 tickers...
Processed 450/501 tickers...
Processed 500/501 tickers...
MOMENTUM QUINTILE DISTRIBUTION

Quintile Distribution:
Momentum_Quintile
1    4797
2    4756
3    4756
4    4762
5    4770
Name: count, dtype: int64

Sample of data with quintiles:
         Date Ticker  Z_Momentum Momentum_Quintile
0  2021-12-01      A    0.476937                 4
1  2021-12-01   AAPL   -0.522008                 2
2  2021-12-01   ABBV   -0.553357                 2
3  2021-12-01    ABT   -0.178020                 3
4  2021-12-01   ACGL   -1.023025                 1
5  2021-12-01    ACN    0.622716                 4
6  2021-12-01   ADBE    0.232228                 4
7  2021-12-01    ADI   -0.247798                 3
8  2021-12-01    ADM  

/var/folders/q5/pzd5gn750_76vhjzjlgj6rfr0000gn/T/ipykernel_62424/392444443.py:134: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  quintile_stats = detailed_df.groupby('Momentum_Quintile')['Forward_Return'].agg([


In [53]:
def calculate_defensive_factor(tickers, sector_map, lookback_years=5, min_daily_obs=40):
    """
    Calculate Low Volatility (Defensive) Factor
    
    Methodology:
    1. Download daily price data
    2. Calculate 60-day realized volatility using log returns
    3. Annualize volatility: sqrt(252) * std(log returns over 60 days)
    4. Invert volatility (lower vol = better): x_low = -sigma
    5. Winsorize across stocks
    6. Calculate sector-neutral Z-scores
    
    Parameters:
    - tickers: List of stock tickers
    - sector_map: Dictionary mapping tickers to sectors (to avoid re-fetching)
    - lookback_years: Years of historical data to fetch
    - min_daily_obs: Minimum daily observations required (default 40 out of 60)
    
    Returns:
    - z_score_matrix: Wide format (Date x Ticker) of defensive Z-scores
    - detailed_df: Long format with all intermediate calculations
    """
    
    
    # 1. FETCH DAILY DATA
    print(f"\nDownloading daily price data for {len(tickers)} tickers...")
    
    try:
        data = yf.download(
            tickers,
            period=f"{lookback_years}y",
            interval="1d",
            progress=True,
            auto_adjust=True,
            threads=True
        )
        
        # Extract Close prices
        if isinstance(data.columns, pd.MultiIndex):
            if 'Close' in data.columns.get_level_values(0):
                prices = data['Close']
            else:
                print("Error: Could not find 'Close' prices")
                return None, None
        else:
            if 'Close' in data.columns:
                prices = data['Close']
            else:
                prices = data
                
    except Exception as e:
        print(f"Error downloading data: {e}")
        return None, None
    
    print(f"Data shape: {prices.shape}")
    
    # 2. CALCULATE DAILY LOG RETURNS
    print("\nCalculating daily log returns...")
    log_returns = np.log(prices / prices.shift(1))
    
    # 3. CALCULATE 60-DAY ROLLING VOLATILITY (ANNUALIZED)
    print("Calculating 60-day rolling realized volatility...")
    window = 60
    
    # Rolling standard deviation of log returns
    rolling_vol = log_returns.rolling(window=window, min_periods=min_daily_obs).std()
    
    # Annualize: multiply by sqrt(252 trading days)
    annualized_vol = rolling_vol * np.sqrt(252)
    
    # 4. RESAMPLE TO MONTHLY (END OF MONTH)
    print("Resampling to monthly frequency...")
    monthly_vol = annualized_vol.resample('M').last()
    
    # Drop tickers with insufficient data
    monthly_vol = monthly_vol.dropna(axis=1, thresh=12)
    valid_tickers = monthly_vol.columns.tolist()
    
    print(f"Valid tickers with sufficient data: {len(valid_tickers)}")
    
    if len(valid_tickers) == 0:
        print("ERROR: No valid tickers with sufficient data!")
        return None, None
    
    # 5. INVERT VOLATILITY (LOWER VOL IS BETTER)
    # Define x_low = -sigma
    low_vol_signal = -monthly_vol
    
    # Drop rows with all NaN
    low_vol_signal = low_vol_signal.dropna(how='all')
    
    # 6. PREPARE FOR CROSS-SECTIONAL STANDARDIZATION
    # Convert to long format
    df_long = low_vol_signal.stack().reset_index()
    df_long.columns = ['Date', 'Ticker', 'Low_Vol_Raw']
    
    # Add original volatility for reference
    vol_long = monthly_vol.stack().reset_index()
    vol_long.columns = ['Date', 'Ticker', 'Realized_Vol']
    df_long = df_long.merge(vol_long, on=['Date', 'Ticker'], how='left')
    
    # Map Sectors (reuse sector map from momentum calculation)
    print("\nMapping sectors from existing sector map...")
    df_long['Sector'] = df_long['Ticker'].map(sector_map)
    
    # 7. WINSORIZATION (Cross-sectional per Date)
    print("Applying winsorization...")
    def winsorize_group(group):
        lower = group.quantile(0.05)
        upper = group.quantile(0.95)
        return group.clip(lower, upper)
    
    df_long['Low_Vol_Winsorized'] = df_long.groupby('Date')['Low_Vol_Raw'] \
                                           .transform(winsorize_group)
    
    # 8. SECTOR Z-SCORES
    print("Calculating sector-neutral Z-scores...")
    def calc_zscore(group):
        if len(group) < 2:
            return 0.0
        sigma = group.std()
        if sigma == 0:
            return 0.0
        return (group - group.mean()) / sigma
    
    df_long['Z_LowVol'] = df_long.groupby(['Date', 'Sector'])['Low_Vol_Winsorized'] \
                                 .transform(calc_zscore)
    
    # Pivot to wide format
    final_factor = df_long.pivot(index='Date', columns='Ticker', values='Z_LowVol')
    
    return final_factor, df_long

# --- EXECUTION ---
# Extract sector map from detailed_df to reuse it
sector_map = detailed_df[['Ticker', 'Sector']].drop_duplicates().set_index('Ticker')['Sector'].to_dict()

print("\nRunning defensive factor calculation...")
lowvol_matrix, defensive_df = calculate_defensive_factor(ticker_list, sector_map)

# Check if calculation was successful
if lowvol_matrix is not None and not lowvol_matrix.empty:
    print("\n" + "="*70)
    print("CALCULATION SUCCESSFUL")
    print("="*70)
    print(f"\nData shape: {defensive_df.shape}")
    print(f"Date range: {defensive_df['Date'].min()} to {defensive_df['Date'].max()}")
    print(f"\nMost Recent Low Volatility Z-Scores (Top 10):")
    latest_date = lowvol_matrix.index[-1]
    print(lowvol_matrix.loc[latest_date].sort_values(ascending=False).head(10))
else:
    print("\nCalculation failed. Please check the errors above and try again.")

[                       1%                       ]  6 of 503 completed


Running defensive factor calculation...



[*********************100%***********************]  503 of 503 completed
/var/folders/q5/pzd5gn750_76vhjzjlgj6rfr0000gn/T/ipykernel_62424/3980232548.py:73: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_vol = annualized_vol.resample('M').last()


Data shape: (1256, 503)

Calculating daily log returns...
Calculating 60-day rolling realized volatility...
Resampling to monthly frequency...
Valid tickers with sufficient data: 501

Mapping sectors from existing sector map...
Applying winsorization...
Calculating sector-neutral Z-scores...

CALCULATION SUCCESSFUL

Data shape: (29353, 7)
Date range: 2021-01-31 00:00:00 to 2025-11-30 00:00:00

Most Recent Low Volatility Z-Scores (Top 10):
Ticker
DAY     1.748777
JNJ     1.657850
BR      1.629342
MCD     1.550804
TJX     1.550804
MSFT    1.493270
LIN     1.421507
ADP     1.403932
PG      1.385116
K       1.385116
Name: 2025-11-30 00:00:00, dtype: float64


In [55]:
# Align dates to Month End to ensure matching with df_global
def align_to_month_end(df):
    df_aligned = df.copy()
    # Convert to period 'M' and back to timestamp 'M' (Month End)
    df_aligned.index = pd.to_datetime(df_aligned.index).to_period('M').to_timestamp('M')
    return df_aligned

# Align the source matrices
z_score_aligned = align_to_month_end(z_score_matrix)
lowvol_aligned = align_to_month_end(lowvol_matrix)

print("Updating df_global with Momentum and Defensive scores...")

# 1. Update Momentum
# Iterate through tickers that exist in both
common_tickers_mom = z_score_aligned.columns.intersection(df_global.columns.get_level_values('ticker').unique())
print(common_tickers_mom)
for ticker in common_tickers_mom:
    # Reindex the source series to match df_global's index
    # This aligns dates and handles any missing/extra dates
    series_aligned = z_score_aligned[ticker].reindex(df_global.index)
    df_global.loc[:, (ticker, "momentum")] = series_aligned

# 2. Update Defensive (Low Vol)
common_tickers_def = lowvol_aligned.columns.intersection(df_global.columns.get_level_values('ticker').unique())

for ticker in common_tickers_def:
    series_aligned = lowvol_aligned[ticker].reindex(df_global.index)
    df_global.loc[:, (ticker, "defensive")] = series_aligned

print("Update complete.")

# Verify
print("\nSample of df_global (Momentum & Defensive):")
# Pick a ticker that likely has data
if len(common_tickers_mom) > 0:
    sample_ticker = common_tickers_mom[30]
    print(f"Ticker: {sample_ticker}")
    print(df_global.loc[:, (sample_ticker, ["momentum", "defensive"])].dropna().head())
else:
    print("No common tickers found.")

Updating df_global with Momentum and Defensive scores...
Index(['A', 'AAPL', 'ABBV', 'ABNB', 'ABT', 'ACGL', 'ACN', 'ADBE', 'ADI', 'ADM',
       ...
       'WY', 'WYNN', 'XEL', 'XOM', 'XYL', 'XYZ', 'YUM', 'ZBH', 'ZBRA', 'ZTS'],
      dtype='object', length=500)
Update complete.

Sample of df_global (Momentum & Defensive):
Ticker: AMT
ticker           AMT          
factor      momentum defensive
2022-02-28 -0.087585  0.039041
2022-03-31 -1.059595 -0.012197
2022-04-30 -1.267119  0.474752
2022-05-31 -1.019808 -0.023576
2022-06-30 -1.401089 -0.227048


In [56]:
from pandas.tseries.offsets import MonthEnd
import pandas as pd

df_quality_value = pd.read_csv("spx_quality_value.csv")

df_quality_value["Date"] = pd.to_datetime(
    df_quality_value["Date"],
    format="%m/%d/%Y"
)

df_quality_value["Date"] = df_quality_value["Date"] + MonthEnd(0)

df_quality_value["EV/EBIT"] = pd.to_numeric(df_quality_value["EV/EBIT"], errors="coerce")
df_quality_value["ROIC"]    = pd.to_numeric(df_quality_value["ROIC"],    errors="coerce")
df_quality_value["EV/EBIT"] = 1.0 / df_quality_value["EV/EBIT"] 
df_quality_value.rename(columns={"EV/EBIT" : "EBIT/EV"})
df_quality_value

,Date,Ticker,EV/EBIT,ROIC
0,2021-12-31,A,0.029270,12.3997
1,2022-01-31,A,0.028335,12.3997
2,2022-02-28,A,0.031962,12.4219
3,2022-03-31,A,0.035243,12.4219
4,2022-04-30,A,0.034802,12.4219
...,...,...,...,...
23836,2025-07-31,ZTS,0.045290,15.0786
23837,2025-08-31,ZTS,0.048661,15.0786
23838,2025-09-30,ZTS,0.047390,15.0786
23839,2025-10-31,ZTS,0.050205,14.4794


In [57]:
df = df_quality_value.copy()
df["Sector"] = df["Ticker"].map(sector_map).fillna("Unknown")

# Decide which columns are factors (everything numeric that is not Date/Ticker/Sector)
exclude_cols = {"Date", "Ticker", "Sector"}
factor_cols = [
    c for c in df.columns
    if c not in exclude_cols and np.issubdtype(df[c].dtype, np.number)
]

print("Factors to normalise:", factor_cols)

# --------- 2) Winsorise cross-sectionally by month ---------
def winsorise_month(group, lower_q=0.01, upper_q=0.99):
    for col in factor_cols:
        p1  = group[col].quantile(lower_q)
        p99 = group[col].quantile(upper_q)
        group[col + "_win"] = group[col].clip(lower=p1, upper=p99)
    return group

df = df.groupby("Date", group_keys=False).apply(winsorise_month)

# --------- 3) Sector z-score within (Date, Sector) ---------
def sector_z_scores(group):
    for col in factor_cols:
        win_col = col + "_win"
        z_col   = col + "_z"

        mu = group[win_col].mean()
        sigma = group[win_col].std(ddof=1)

        if sigma == 0 or np.isnan(sigma):
            group[z_col] = np.nan
        else:
            group[z_col] = (group[win_col] - mu) / sigma
    return group

df = df.groupby(["Date", "Sector"], group_keys=False).apply(sector_z_scores)

# If you only want the normalised (z-scored) values and not the winsorised columns, you can drop them:
# df = df.drop(columns=[c for c in df.columns if c.endswith("_win")])

df_normalised = df
df_normalised.head()


Factors to normalise: ['EV/EBIT', 'ROIC']


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:4671: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:4671: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:4671: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:4671: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:4671: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b

,Date,Ticker,EV/EBIT,ROIC,Sector,EV/EBIT_win,ROIC_win,EV/EBIT_z,ROIC_z
0,2021-12-31,A,0.029270,12.3997,Healthcare,0.029270,12.3997,-0.832565,-0.000117
1,2022-01-31,A,0.028335,12.3997,Healthcare,0.028335,12.3997,-0.784013,-0.048445
2,2022-02-28,A,0.031962,12.4219,Healthcare,0.031962,12.4219,-0.728157,-0.004601
3,2022-03-31,A,0.035243,12.4219,Healthcare,0.035243,12.4219,-0.554917,-0.013044
4,2022-04-30,A,0.034802,12.4219,Healthcare,0.034802,12.4219,-0.566935,0.041856


In [58]:
df_normalised

,Date,Ticker,EV/EBIT,ROIC,Sector,EV/EBIT_win,ROIC_win,EV/EBIT_z,ROIC_z
0,2021-12-31,A,0.029270,12.3997,Healthcare,0.029270,12.3997,-0.832565,-0.000117
1,2022-01-31,A,0.028335,12.3997,Healthcare,0.028335,12.3997,-0.784013,-0.048445
2,2022-02-28,A,0.031962,12.4219,Healthcare,0.031962,12.4219,-0.728157,-0.004601
3,2022-03-31,A,0.035243,12.4219,Healthcare,0.035243,12.4219,-0.554917,-0.013044
4,2022-04-30,A,0.034802,12.4219,Healthcare,0.034802,12.4219,-0.566935,0.041856
...,...,...,...,...,...,...,...,...,...
23836,2025-07-31,ZTS,0.045290,15.0786,Healthcare,0.045290,15.0786,-0.369071,0.518135
23837,2025-08-31,ZTS,0.048661,15.0786,Healthcare,0.048661,15.0786,-0.358546,0.517568
23838,2025-09-30,ZTS,0.047390,15.0786,Healthcare,0.047390,15.0786,-0.358267,0.517568
23839,2025-10-31,ZTS,0.050205,14.4794,Healthcare,0.050205,14.4794,-0.256269,0.439994


In [59]:
df_qv = df_normalised[["Date", "Ticker", "EV/EBIT_z", "ROIC_z"]].copy()
df_qv = df_qv.rename(columns={
    "EV/EBIT_z": "value",
    "ROIC_z": "quality"
})

df_qv = df_qv.set_index(["Date", "Ticker"])

df_qv_wide = df_qv.unstack("Ticker")      # columns: (factor, Ticker)
df_qv_wide = df_qv_wide.swaplevel(axis=1) # now: (Ticker, factor)
df_qv_wide = df_qv_wide.sort_index(axis=1)

# Make sure column names match df_global
df_qv_wide.columns.set_names(["ticker", "factor"], inplace=True)

# 4) Update df_global in place (only matching index/columns are filled)
df_global.update(df_qv_wide)

# df_global now has quality & value filled where available
df_global.head()


ticker           MMM                                     AOS            \
factor      momentum defensive   quality     value  momentum defensive   
2022-02-28 -0.952279  0.833603 -0.222626  0.891645  1.020780  0.547651   
2022-03-31 -1.303795  0.881599 -0.230946  1.134598  0.520822  0.684724   
2022-04-30 -1.623701  1.187277 -0.372789  1.000279  0.199430  0.567785   
2022-05-31 -1.431006  1.162275 -0.369093  0.828876 -0.523985  0.312249   
2022-06-30 -1.171542  1.117635 -0.365133  0.667430 -0.735469  0.371016   

ticker                               ABT            ...      ZBRA            \
factor       quality     value  momentum defensive  ...   quality     value   
2022-02-28  0.704285 -0.045489 -0.427058  0.607635  ...  0.612203 -0.312895   
2022-03-31  0.695043  0.184541 -0.351186  0.481921  ...  0.589274 -0.130660   
2022-04-30  0.798980  0.240995 -0.330217  0.512631  ...  0.641488 -0.118717   
2022-05-31  0.810383  0.588815 -0.390436  0.535799  ...  0.575922 -0.064808   
2022-06-30  0.811978  0.379333 -0.027111  0.575290  ...  0.576111 -0.067514   

ticker           ZBH                                     ZTS            \
factor      momentum defensive   quality     value  momentum defensive   
2022-02-28 -1.805453 -0.389828 -0.985093 -0.426869  1.317687  0.475576   
2022-03-31 -1.957289 -0.179308 -0.968841 -0.482919  0.905417  0.326307   
2022-04-30 -1.805852 -0.345537 -1.003570 -0.494140  0.300170  0.539837   
2022-05-31 -1.685474  0.370927 -1.004145 -0.468246 -0.143425  0.303216   
2022-06-30 -1.099072  0.302245 -1.004145 -0.420942 -0.204897  0.394111   

ticker                          
factor       quality     value  
2022-02-28  0.119167 -0.842493  
2022-03-31  0.107607 -0.671347  
2022-04-30  0.149127 -0.685685  
2022-05-31  0.153191 -0.607311  
2022-06-30  0.153191 -0.524816  

[5 rows x 2012 columns]

In [60]:
# Calculate the composite signal by averaging the 4 factors
# We take the mean across the 'factor' level (level 1) of the columns
# This will result in a DataFrame with Date index and Ticker columns
df_signal = df_global.groupby(level='ticker', axis=1).mean()

print("Signal DataFrame created.")
print(f"Shape: {df_signal.shape}")
print("\nSample of df_signal (First 5 rows, first 5 columns):")
print(df_signal.iloc[:5, :5])

# Optional: Check for missing values
print(f"\nTotal missing values: {df_signal.isna().sum().sum()}")
print(f"Percentage of missing values: {df_signal.isna().sum().sum() / df_signal.size * 100:.2f}%")

Signal DataFrame created.
Shape: (47, 503)

Sample of df_signal (First 5 rows, first 5 columns):
ticker             A      AAPL      ABBV      ABNB       ABT
2022-02-28 -0.127307  1.012971  0.155386 -1.536406 -0.195656
2022-03-31 -0.216669  1.241705  0.324375 -1.453116 -0.181021
2022-04-30 -0.299115  1.041464  0.438741 -0.854234 -0.120997
2022-05-31 -0.361502  1.052559  0.527099 -0.330017 -0.110353
2022-06-30 -0.395865  0.975333  0.454667 -0.557684 -0.006773

Total missing values: 700
Percentage of missing values: 2.96%


/var/folders/q5/pzd5gn750_76vhjzjlgj6rfr0000gn/T/ipykernel_62424/3437236992.py:4: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_signal = df_global.groupby(level='ticker', axis=1).mean()


In [61]:
df_signal

ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XYL,XYZ,YUM,ZBH,ZBRA,ZTS
2022-02-28,-0.127307,1.012971,0.155386,-1.536406,-0.195656,0.623736,0.531385,-0.241347,-0.521679,0.309196,...,1.602212,-1.166239,-0.105714,0.353231,-0.770877,-1.438584,0.670070,-0.901811,0.029246,0.267484
2022-03-31,-0.216669,1.241705,0.324375,-1.453116,-0.181021,0.678098,0.460009,-0.079461,-0.496106,0.529642,...,1.729318,-1.360486,-0.034978,0.373947,-0.733847,-1.393891,0.634203,-0.897089,0.051831,0.166996
2022-04-30,-0.299115,1.041464,0.438741,-0.854234,-0.120997,0.772569,0.290698,-0.175692,-0.428604,0.352258,...,1.792670,-1.304335,-0.155388,0.412694,-0.938688,-1.412717,0.651058,-0.912275,-0.108674,0.075862
2022-05-31,-0.361502,1.052559,0.527099,-0.330017,-0.110353,0.821432,0.367515,-0.150169,-0.382158,0.485232,...,1.724196,-1.262309,-0.282241,0.555465,-0.824677,-1.484489,0.586041,-0.696735,-0.126761,-0.073582
2022-06-30,-0.395865,0.975333,0.454667,-0.557684,-0.006773,0.888284,0.308784,-0.275211,-0.290085,0.388827,...,2.221921,-1.344943,0.016659,0.461744,-0.864435,-1.502930,0.732930,-0.555478,-0.141941,-0.045603
2022-07-31,-0.363045,0.820666,0.478964,-0.562283,0.082782,0.927540,0.249978,-0.242465,-0.046059,0.596513,...,1.961747,-1.307728,-0.021649,0.402168,-0.845695,-1.528028,0.574774,-0.472484,-0.277573,-0.174815
2022-08-31,-0.441905,0.957951,0.538692,-0.812688,0.085158,0.949566,0.353750,-0.254291,-0.098055,0.287564,...,1.667380,-1.227345,0.016689,0.469000,-0.793531,-1.572457,0.627342,-0.423048,-0.631948,-0.105151
2022-09-30,-0.350777,0.962447,0.404104,-0.694211,0.092844,0.830100,0.352652,-0.587996,-0.118278,0.465826,...,1.437912,-1.167530,0.040921,0.475760,-0.809915,-1.598626,0.708101,-0.490909,-0.624190,-0.053424
2022-10-31,-0.162012,0.878128,0.378747,-0.517082,-0.073885,0.383310,0.282475,-0.567111,-0.217282,0.433093,...,1.840742,-1.307877,0.050808,0.558205,-0.761430,-1.572058,0.421006,-0.342251,-0.580365,-0.097469
2022-11-30,-0.198025,0.700830,0.431231,-0.623035,-0.029155,0.534694,0.214909,-0.498954,-0.200225,0.759891,...,1.496352,-0.946384,0.011149,0.650951,-0.595039,-1.385593,0.549362,-0.212407,-0.656296,-0.408472
